Get packages

In [1]:
import os

os.chdir(os.path.join(os.getcwd(), '..'))
import src.preprocessing
import src.model_training
import src.feature_engineering

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import time
import shap
import lightgbm as lgb

import json
import sys
import pickle

from verstack import LGBMTuner
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Get data & model

In [2]:
df = src.feature_engineering.get_train_data('data/data_pricing_challenge.csv')
variables = src.model_training.txt_load('utils/model_variables.txt')
df = df.loc[df['price'] >= 20000]

In [3]:
pkl_file_path  = 'models/model'
# Load the model
with open(pkl_file_path, 'rb') as f:
    model = pickle.load(f)

Approach 1: Selling price is known

In [11]:
df_pred = df[variables]
df_pred['mileage'] = df_pred['mileage']+10000
df_pred['antiquity'] = df_pred['antiquity']+1


# Make predictions
prediction = model.predict(df_pred)

# Add prediction to dataset
df_pred['price_pred_fin'] = prediction
df_pred['price_real_ini'] = df['price']

df_pred['loss_in_value'] = df_pred['price_real_ini'] - df_pred['price_pred_fin']
df_pred = df_pred.sort_values(by='loss_in_value', ascending = True)
df_pred.head()

,mileage,engine_power,antiquity,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,convertible,coupe,estate,hatchback,sedan,subcompact,suv,van,avg_price,avg_mileage,avg_antiquity,avg_price_per_milleage,avg_price_per_antiquity,price_pred_fin,price_real_ini,loss_in_value
4583,63055,140,4.085558,1,1,0,0,1,0,1,1,0,0,0,0,0,0,1,0,31475.56,80901.56,3.13,0.3891,10056.0895,30109.373036,23800,-6309.373036
3074,92647,230,4.663244,1,1,1,0,0,0,1,1,0,0,0,0,1,0,0,0,37772.22,126482.94,4.85,0.2986,7788.0866,38810.958936,32600,-6210.958936
4305,144154,300,8.167693,1,1,0,1,1,0,1,1,0,0,0,0,0,0,1,0,34461.36,124492.84,5.76,0.2768,5982.8750,27721.779966,22000,-5721.779966
3873,10706,155,1.588638,1,1,0,1,1,1,1,1,0,0,0,0,0,0,1,0,30570.13,130993.32,5.64,0.2334,5420.2358,46651.795052,41600,-5051.795052
3543,50142,280,2.998631,1,1,1,1,1,1,1,1,0,0,0,0,1,0,0,0,34900.00,141101.33,4.20,0.2473,8309.5238,45698.338234,41100,-4598.338234


In [17]:
original_data = src.preprocessing.load_data('data/data_pricing_challenge.csv')

In [19]:
original_data.loc[(original_data['mileage'] == 53055)]

,maker_key,model_key,mileage,engine_power,registration_date,fuel,paint_color,car_type,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,price,sold_at
4648,BMW,X4,53055,140,07/01/2015,diesel,black,suv,True,True,False,False,True,False,True,True,23800,08/01/2018


## Conclusions

nmaxm